# Ackerberg-Keane-Wolpin importance sampling weights

In the last workbook, we assembled some hypothetical shares, error terms (for viewership and price) and upper bounds on error terms under the assumption that observed programming sequences were Nash equilibria. Now, we have to assemble the results into importance sampling weights. 

The difficult part of the importance sampling scheme is that we have to check for multiplicity of equilibria - this is the cohesiveness problem that everyone talks about. 


### Refresher - the basic idea:

Just so that we can be sure we have a firm idea what we are doing, let's take a little time to explain things. We require that the unobserved errors (which have idiosyncratic, market-time, and station components) must fall into some region so that they aren't large enough to conflict with the idea that observed strategies are Nash equilibria. While we had to assemble the errors sequentially, let's put that aside for the moment and just observe that there is some upper bound on the errors. Anyways:

Starting with the price equation, we have the likelihood that $p' \leq p$ (or $\ln p \leq \ln p$ ) as: 

$$
\textrm{Prob}(\ln p' < \ln p) = \int_{e_p,e_v \in \Omega} \left( \ln Ms'(X_v\beta_v+e_v) + X\beta_p + e_p \right)f_p(e_p)f_v(e_v)de_pde_v
$$

where $\Omega$ is some difficult region to integrate. To get an AKW sampler, we want to create simulated individuals, and reweight these in estimation. To do this, we first do a change of variables to a $u$ term, which in practice amounts to constituting linear terms. So, if $e_v$ is $N(0,\sigma_v)$, then $u_v = X_v\beta_v+e_v$ is $N(X_v\beta_v,\sigma_v)$. In terms of the above, we get something like:

$$
\int_{u_p,u_v \in \Omega'} h_{p|v}(u_p;\eta Ms(u_v)+X_p\beta_p,\sigma_p)h_v(s;X_v\beta_v,\sigma_v)du_vdu_p
$$

Now, if we have a preliminary model, and can get a sample of $u$ terms from some density $g(u)$, we have:

$$
\int_{u_p,u_v \in \Omega'} h_{p|v}(u_p;\eta Ms(u_v)+X_p\beta_p,\sigma_p)h_v(s;X_v\beta_v,\sigma_v)\frac{g_{p|v}(u_p)g_v(u_v)}{g_{p|v}(u_p)g_v(u_v)}du_vdu_p
$$

So, we can simulate the above difficult integral with:

$$
\approx \frac{1}{S}\sum\frac{h_{p|v}(u_p;\eta Ms(u_v)+X_p\beta_p,\sigma_p)h_v(s(u_v);X_v\beta_v,\sigma_v)}{g_{p|v}(u_p)g_v(u_v)}
$$


### The genius of it all...

If one looks carefully at the above, we now have a sample of hypothetical utility indices, and viewership shares, etc. Conditional on having these, everything else is linear - linear viewership utility index and conditional on simulated shares, linear pricing. So, given the error terms, estimation is actually pretty straightforward. 

### The one remaining problem...

There is one difficulty - we have a pricing equation conditional on a distribution of a hypothetical share, which depends upon a viewership index $u$, in a way that is a bit nuanced. So, let's ask: what is the sampling density of the hypothetical share? 

To recover this, start with the following equation:

and 

$$ 
\ln s' -\ln s_0' =\mu' \ln s'_{ig} +X\beta_v+u_s+u_{mt}+e_{imt} 
$$ 
 
Since simulating and dealing with the random effects is easy, let's fold these all into one thing: 

$$ 
\ln s' -\ln s_0' =\mu' \ln s'_{ig} +u_v 
$$ 

and observe that our Berry (1994) share model is
 

$$
s' = \frac{e^{\frac{U_i}{1-\mu}}}{\sum_g e^{\frac{U_k}{1-\mu}}}\frac{(\sum_g e^{\frac{U_k}{1-\mu}})^{1-\mu}}{\sum_G(\sum e^{\frac{U_k}{1-\mu}})^{1-\mu}}
$$

Since $u_v$ has a simple normal density, we ask, given this, what is the density of $\ln s'$, the simulated share? The old trick, heuristically, is:

$$
f(\ln s)d\ln s=g(u)du
$$

or

$$
f(\ln s)=g(u)|\frac{du}{d\ln s}|
$$

or

$$
f(\ln s)=g(u)\frac{1}{|\frac{d\ln s}{du}|}
$$

So, our question now becomes: what is the derivative of the log share with respect to $u$? 

So, taking logs of the right hand side of the above gives:


$$
\ln s' = \frac{U_i}{1-\mu}-\mu \ln\left(\sum_g e^{\frac{U_k}{1-\mu}}\right)-\ln \sum_G(\sum e^{\frac{U_k}{1-\mu}})^{1-\mu}
$$

Taking the derivative gives:

$$
\frac{d \ln s'}{dU_i}=\frac{1}{1-\mu}-\frac{\mu}{1-\mu}\frac{e^{\frac{U_i}{1-\mu}}}{\sum_g e^{\frac{U_k}{1-\mu}}}-e^{\frac{U_i}{1-\mu}}\frac{(\sum_g e^{\frac{U_k}{1-\mu}})^{-\mu}}{\sum_G(\sum e^{\frac{U_k}{1-\mu}})^{1-\mu}}
$$

If we multiply and divide the last term by $\sum_g e^{\frac{U_k}{1-\mu}} $, we get:

$$
\frac{d \ln s'}{dU_i}=\frac{1}{1-\mu}-\frac{\mu}{1-\mu}\frac{e^{\frac{U_i}{1-\mu}}}{\sum_g e^{\frac{U_k}{1-\mu}}}-\frac{e^{\frac{U_i}{1-\mu}}}{\sum_g e^{\frac{U_k}{1-\mu}}}\frac{(\sum_g e^{\frac{U_k}{1-\mu}})^{1-\mu}}{\sum_G(\sum e^{\frac{U_k}{1-\mu}})^{1-\mu}}
$$

Now, we see that what we have is a bunch of terms that are effectively simulated shares. The second term on the right is the simulated within-group share, while the third term on the right is the simulated actual share. Hence:

$$
\frac{d \ln s'}{dU_i}=\frac{1}{1-\mu}-\frac{\mu}{1-\mu}s_{i|g}'-s_{i|g}'s_g
$$

This is pretty easy to translate into the required jacobian term, so we can use it in our density. Once again, it is nice that all of the right-hand-side terms are functions of $u_v$, so the AKW sampler works!

Let's get the programming rolling...As usual, we first have to import the requisite packages...




In [14]:
import ipystata
import os
os.getcwd()

'C:\\Users\\mjbaker\\Documents\\GitHub\\NewsHour'

In [15]:
%%stata -s nh1
cd C:\Users\mjbaker\Documents\GitHub\NewsHour
clear all
use AveragedDataDyno.dta
set more off


C:\Users\mjbaker\Documents\GitHub\NewsHour


Now, in the last notebook, we got simulated shares, error terms, and bounds on the errors (really, the recursively bounded errors). These will all figure into computing our importance sampling density, so let's get these going to begin with:

In [16]:
%%stata -s nh1 --mata
mata matuse PVDraws

Mata output:

(loading priceBounds, priceErrs, sharesBCS, viewBounds, viewErrs)


While the following command is once again repetitive, I take comfort in knowing exactly how this variable is generated, and exactly how the data is sorted:

In [17]:
%%stata -s nh1
gen game= (lnews | otherl) & lnpps!=.
sort market stationid mt timeslot

We also need all of our data to be read in and reshaped. I know this is getting tedious...and there is still more to be done!

In [18]:
%%stata -s nh1 --mata
    st_view(lnews=.,.,"lnews")
    st_view(nnews=.,.,"nnews")
    st_view(otherl=.,.,"otherl")
    st_view(otherc=.,.,"otherc")
    st_view(si=.,.,"si")
    st_view(mt=.,.,"mt")
    st_view(id=.,.,"stationid")
    st_view(M=.,.,"market")
    st_view(time=.,.,"timeslot")
    st_view(game=.,.,"game")
    st_view(lnpps=.,.,"lnpps")
    st_view(pop=.,.,"ACS_HH") 
    st_view(l_ACS_HH=.,.,"l_ACS_HH")
    st_view(lnewsn=.,.,"lnewsn")
    st_view(otherln=.,.,"otherln")
    st_view(nnewsn=.,.,"nnewsn")
    st_view(othercn=.,.,"othercn")
    st_view(dln=.,.,"dln")

    lnviewn=ln(pop:*si)
    m=panelsetup(M,1)

Mata output:

:     st_view(lnews=.,.,"lnews")

:     st_view(nnews=.,.,"nnews")

:     st_view(otherl=.,.,"otherl")

:     st_view(otherc=.,.,"otherc")

:     st_view(si=.,.,"si")

:     st_view(mt=.,.,"mt")

:     st_view(id=.,.,"stationid")

:     st_view(M=.,.,"market")

:     st_view(time=.,.,"timeslot")

:     st_view(game=.,.,"game")

:     st_view(lnpps=.,.,"lnpps")

:     st_view(pop=.,.,"ACS_HH") 

:     st_view(l_ACS_HH=.,.,"l_ACS_HH")

:     st_view(lnewsn=.,.,"lnewsn")

:     st_view(otherln=.,.,"otherln")

:     st_view(nnewsn=.,.,"nnewsn")

:     st_view(othercn=.,.,"othercn")

:     st_view(dln=.,.,"dln")

:     m=panelsetup(M,1)


The usual mata reshaping routine:

In [19]:
%%stata -s nh1 -np
do MataFunctions\MataReshape.do

The following creates placeholders for everything we need to assemble an importance sampler. Here goes:

In [20]:
%%stata -s nh1 --mata
    uv      = J(rows(statIdLong),0,.)
    uvg     = J(rows(statIdLong),0,.)
    uvb     = J(rows(statIdLong),0,.)
    uvsi    = J(rows(statIdLong),0,.)
    uvsg    = J(rows(statIdLong),0,.)
    uvre1   = J(rows(statIdLong),0,.)
    uvre2   = J(rows(statIdLong),0,.)
    uvre1g  = J(rows(statIdLong),0,.)
    uvre2g  = J(rows(statIdLong),0,.)
    up      = J(rows(statIdLong),0,.)
    upg     = J(rows(statIdLong),0,.)
    upb     = J(rows(statIdLong),0,.)
    upre1   = J(rows(statIdLong),0,.)
    upre2   = J(rows(statIdLong),0,.)
    upre1g  = J(rows(statIdLong),0,.)
    upre2g  = J(rows(statIdLong),0,.)
    looper  = uniqrows(asarray_keys(Bcs)[,1])
    counter = 1

Mata output:

:     uv      = J(rows(statIdLong),0,.)

:     uvg     = J(rows(statIdLong),0,.)

:     uvb     = J(rows(statIdLong),0,.)

:     uvsi    = J(rows(statIdLong),0,.)

:     uvso    = J(rows(statIdLong),0,.)

:     uvsg    = J(rows(statIdLong),0,.)

:     uvre1   = J(rows(statIdLong),0,.)

:     uvre2   = J(rows(statIdLong),0,.)

:     uvre1g  = J(rows(statIdLong),0,.)

:     uvre2g  = J(rows(statIdLong),0,.)

:     up      = J(rows(statIdLong),0,.)

:     upg     = J(rows(statIdLong),0,.)

:     upb     = J(rows(statIdLong),0,.)

:     upre1   = J(rows(statIdLong),0,.)

:     upre2   = J(rows(statIdLong),0,.)

:     upre1g  = J(rows(statIdLong),0,.)

:     upre2g  = J(rows(statIdLong),0,.)

:     looper  = uniqrows(asarray_keys(Bcs)[,1])
                 <istmt>:  3499  Bcs not found
-----------------------------------------------------------------------------------------------------------------------------------------------------------------
r(3499);
unrecognized command:  

In [22]:
%%stata -s nh1
exit

Please note: session nh1 was no longer active and is restarted.


AttributeError: stata.StataOLEApp.UtilIsStataFree